In [1]:
from google.colab import files
import zipfile
import sqlite3
import pandas as pd
import os

uploaded = files.upload()

Saving Approach to Complex SQLquery Building in Kaggle (4).zip to Approach to Complex SQLquery Building in Kaggle (4).zip


In [ ]:
zip_filename = list(uploaded.keys())[0]
print(f"Uploaded zip file: {zip_filename}")

extract_dir = "/content/extracted"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print(f"Extracted files: {zip_ref.namelist()}")

extracted_files = os.listdir(extract_dir)
print("\nExtracted files:")
for file in extracted_files:
    print(f"- {file}")

In [ ]:
db_files = [f for f in extracted_files if f.lower().endswith(('.sqlite', '.db', '.sqlite3'))]

if not db_files:

    all_files = os.listdir(extract_dir)
    print("\nNo obvious database file found. All extracted files:")
    for file in all_files:
        print(f"- {file}")

    for file in all_files:
        file_path = os.path.join(extract_dir, file)
        if os.path.isfile(file_path):
            try:
                test_conn = sqlite3.connect(file_path)
                cursor = test_conn.cursor()
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
                tables = cursor.fetchall()
                if tables:
                    db_files = [file]
                    print(f"\nFound potential database file: {file}")
                    print(f"Contains {len(tables)} tables")
                    break
            except sqlite3.DatabaseError:
                pass

if not db_files:
    raise FileNotFoundError("No SQLite database file found in the zip archive")

db_filename = os.path.join(extract_dir, db_files[0])
print(f"\nUsing database file: {db_filename}")

conn = sqlite3.connect(db_filename)

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nTables in the database:")
for table in tables:
    table_name = table[0]
    print(f"\nTable: {table_name}")

    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()

    print("Columns:")
    for col in columns:
        print(f"  - {col[1]} ({col[2]})")

In [ ]:
config = {
    'player_table': 'Player',
    'ball_table': 'Ball_by_Ball',
    'player_id_col': 'Player_Id',
    'player_name_col': 'Player_Name',
    'runs_col': 'Runs_Scored',
    'striker_col': 'Striker',
    'bowler_col': 'Bowler',
    'wicket_col': 'Out_Type',
    'match_id_col': 'Match_Id'
}

player_table = config['player_table']
ball_table = config['ball_table']
player_id = config['player_id_col']
player_name = config['player_name_col']
runs_col = config['runs_col']
striker_col = config['striker_col']
bowler_col = config['bowler_col']
wicket_col = config['wicket_col']
match_id = config['match_id_col']

In [ ]:
def run_query(conn, query, title):
    try:
        result = pd.read_sql(query, conn)
        print(f"\n{title}")
        print(result.head(10))
        return True
    except Exception as e:
        print(f"\nError in {title}: {str(e)}")
        return False

In [ ]:
query1 = f"SELECT * FROM {player_table} LIMIT 5"
run_query(conn, query1, "Query 1: Player Table Sample")

In [ ]:
query2 = f"""
SELECT
    p.{player_id} AS player_id,
    p.{player_name} AS player_name,
    SUM(b.{runs_col}) AS total_runs
FROM {ball_table} b
JOIN {player_table} p ON b.{striker_col} = p.{player_id}
GROUP BY p.{player_id}, p.{player_name}
ORDER BY total_runs DESC
LIMIT 10
"""
run_query(conn, query2, "Query 2: Top Batsmen by Runs")

In [ ]:
query3 = f"""
SELECT
    p.{player_id},
    p.{player_name},
    SUM(CASE WHEN inning_runs >= 50 AND inning_runs < 100 THEN 1 ELSE 0 END) AS fifties,
    SUM(CASE WHEN inning_runs >= 100 THEN 1 ELSE 0 END) AS hundreds
FROM (
    SELECT
        {striker_col} AS player_id,
        {match_id},
        SUM({runs_col}) AS inning_runs
    FROM {ball_table}
    GROUP BY {striker_col}, {match_id}
) innings
JOIN {player_table} p ON innings.player_id = p.{player_id}
GROUP BY p.{player_id}, p.{player_name}
ORDER BY hundreds DESC, fifties DESC
LIMIT 10
"""
run_query(conn, query3, "Query 3: Fifties and Hundreds")

In [ ]:
query4 = f"""
SELECT
    p.{player_id},
    p.{player_name},
    MAX(wickets) AS best_wickets,
    MIN(runs) AS best_runs
FROM (
    SELECT
        {bowler_col} AS player_id,
        {match_id},
        COUNT(wt.Kind_Out) AS wickets,  -- Assuming Kind_Out indicates a wicket
        SUM(bs.Runs_Scored) AS runs
    FROM {ball_table} bb
    LEFT JOIN Wicket_Taken wt ON bb.Match_Id = wt.Match_Id AND bb.Over_Id = wt.Over_Id AND bb.Ball_Id = wt.Ball_Id AND bb.Innings_No = wt.Innings_No
    LEFT JOIN Batsman_Scored bs ON bb.Match_Id = bs.Match_Id AND bb.Over_Id = bs.Over_Id AND bb.Ball_Id = bs.Ball_Id AND bb.Innings_No = bs.Innings_No
    GROUP BY {bowler_col}, {match_id}
) innings
JOIN {player_table} p ON innings.player_id = p.{player_id}
GROUP BY p.{player_id}, p.{player_name}
ORDER BY best_wickets DESC, best_runs ASC
LIMIT 10
"""
run_query(conn, query4, "Query 4: Best Bowling Figures")

In [ ]:
query5 = f"""
WITH BattingStats AS (
    SELECT
        {striker_col} AS player_id,
        COUNT(DISTINCT {match_id}) AS matches,
        SUM({runs_col}) AS total_runs,
        ROUND(CAST(SUM({runs_col}) AS REAL) / NULLIF(COUNT(DISTINCT {match_id}), 0), 2) AS batting_avg,
        MAX({runs_col}) AS highest_score,
        SUM(CASE WHEN inning_runs >= 50 AND inning_runs < 100 THEN 1 ELSE 0 END) AS fifties,
        SUM(CASE WHEN inning_runs >= 100 THEN 1 ELSE 0 END) AS hundreds
    FROM (
        SELECT
            {striker_col},
            {match_id},
            SUM({runs_col}) AS inning_runs
        FROM {ball_table}
        GROUP BY {striker_col}, {match_id}
    ) innings
    GROUP BY {striker_col}
),
BowlingStats AS (
    SELECT
        {bowler_col} AS player_id,
        SUM(CASE WHEN wt.Kind_Out IS NOT NULL THEN 1 ELSE 0 END) AS total_wickets,
        ROUND(CAST(SUM(bs.Runs_Scored) AS REAL) / NULLIF(SUM(CASE WHEN wt.Kind_Out IS NOT NULL THEN 1 ELSE 0 END), 0), 2) AS bowling_avg,
        MAX(wickets) AS best_wickets,
        MIN(runs) AS best_runs
    FROM (
        SELECT
            bb.{bowler_col},
            bb.{match_id},
            COUNT(wt.Kind_Out) AS wickets,
            SUM(bs.Runs_Scored) AS runs
        FROM {ball_table} bb
        LEFT JOIN Wicket_Taken wt ON bb.Match_Id = wt.Match_Id AND bb.Over_Id = wt.Over_Id AND bb.Ball_Id = wt.Ball_Id AND bb.Innings_No = bb.Innings_No
        LEFT JOIN Batsman_Scored bs ON bb.Match_Id = bs.Match_Id AND bb.Over_Id = bs.Over_Id AND bb.Ball_Id = bs.Ball_Id AND bb.Innings_No = bb.Innings_No
        GROUP BY bb.{bowler_col}, bb.{match_id}
    ) innings
    GROUP BY player_id
)
SELECT
    p.{player_id},
    p.{player_name},
    COALESCE(b.matches, 0) AS matches,
    COALESCE(b.total_runs, 0) AS total_runs,
    COALESCE(b.batting_avg, 0) AS batting_avg,
    COALESCE(b.highest_score, 0) AS highest_score,
    COALESCE(b.fifties, 0) AS fifties,
    COALESCE(b.hundreds, 0) AS hundreds,
    COALESCE(bw.total_wickets, 0) AS total_wickets,
    COALESCE(bw.bowling_avg, 0) AS bowling_avg,
    COALESCE(bw.best_wickets, 0) || '/' || COALESCE(bw.best_runs, 0) AS best_bowling
FROM {player_table} p
LEFT JOIN BattingStats b ON p.{player_id} = b.player_id
LEFT JOIN BowlingStats bw ON p.{player_id} = bw.player_id
ORDER BY total_runs DESC
LIMIT 10
"""
run_query(conn, query5, "Query 5: Comprehensive Career Metrics")

conn.close()